* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/ec2-user/SageMaker/just-a-recipe-generator/notebooks'

In [3]:
# Local
# %cd ~/Documents/github/just-a-recipe-generator

# SageMaker
%cd ~/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [4]:
## Local - mac (installing tf was a painnnn)
# !pip install nltk

In [5]:
## SageMaker
# !pip install tensorflow==2.8.0

In [6]:
import os
import time
import copy
import tensorflow as tf

In [7]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids, prep_data
                                            )

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
from src.models.rnn import train_rnn_model, apply_one_step_model, OneStep

In [9]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [10]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [11]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [ ]:
# """Convert dict of recipes into a string"""
# all_ingredients = []

# for recipe in list(recipe_dict.keys()):
#     all_ingredients.extend([recipe])
#     all_ingredients.extend(["Ingredients:"])
#     all_ingredients.extend(recipe_dict[recipe])

# ingredients_text = ""

# for ingredient in all_ingredients:
# #     if ingredient == "Ingredients:":
# #         ingredients_text += "\n"
#     ingredient_words = ingredient.split(" ")
#     ingredients_text += ("\n " + " ".join(ingredient_words))

# ingredients_text = ingredients_text.lstrip()
# print(ingredients_text[:100]

In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
dataset, chars_from_ids, ids_from_chars, vocab = prep_data(recipe_dict)

2023-04-12 20:58:23.799245: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [ ]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# text = ingredients_to_text(recipe_dict)
# vocab, ids_from_chars, chars_from_ids = tokenize_text(text)

In [13]:
def create_one_step_model(model, chars_from_ids, ids_from_chars):
    """Run the one-step RNN model in a loop to generate text."""
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    return one_step_model


def apply_one_step_model(one_step_model, steps=1000):
    start = time.time()
    states = None
#     next_char = tf.constant(['Ingredients:'])
    next_char = tf
    result = [next_char]

    for n in range(steps):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()
    print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
    print('\nRun time:', end - start)

    return one_step_model


def save_model(model, model_name="one_step"):
    tf.saved_model.save(model, model_name)
    return None


def retrieve_model():
    saved_model = tf.saved_model.load(
        "one_step", tags=None, options=None
    )
    return saved_model

In [ ]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=25)
apply_one_step_model(model)

In [15]:
# to reuse model
model = retrieve_model()

In [18]:
one_step_model = create_one_step_model(model, chars_from_ids, ids_from_chars)
apply_one_step_model(model)

TypeError: Exception encountered when calling layer 'string_lookup_1' (type StringLookup).

Cannot convert ['[UNK]'] to EagerTensor of dtype int64

Call arguments received by layer 'string_lookup_1' (type StringLookup):
  • inputs=["'[UNK]'"]

In [ ]:
ids_from_chars(['[UNK]'])

In [ ]:
apply_one_step_model(model)

In [ ]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=75)
apply_one_step_model(model, chars_from_ids, ids_from_chars)

In [ ]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

In [ ]:
saved_model = tf.saved_model.load(
    "one_step", tags=None, options=None
)

In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

# EVERYTHING BELOW IS JUST TESTING

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit(dataset, epochs=1)